# VQE 
## 1. State preparation circuit
Convert |000000> state to an actual state |$\phi(\theta)$> with an arbitary input parameters set $\theta$

In [1]:
# Define and verify ansatz
import math, random
from typing import Callable, List, Tuple, Optional
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector



def _build_hf_init(n_qubits: int, n_particles: int):
    try:
        from qiskit import QuantumCircuit
        qc = QuantumCircuit(n_qubits, name="HF")
        i = 0
        while i < n_particles:
            qc.x(i); i += 1
        return qc
    except Exception as e:
        raise RuntimeError(f"HF init build failed: {e}")

def _statevector_from_circuit(qc):
    try:
        from qiskit.quantum_info import Statevector
        return Statevector.from_instruction(qc)
    except Exception as e:
        print(f"[ERR] Statevector simulation failed: {e}")
        return None

def _state_distance(u, v) -> float:
    if u is None or v is None or len(u) != len(v): return float("inf")
    s = 0.0; i = 0
    while i < len(u):
        d = u[i] - v[i]
        s += (d.real*d.real + d.imag*d.imag)
        i += 1
    return math.sqrt(s)

def _bit(z: int, i: int) -> int:
    return (z >> i) & 1

def _z_exp_from_state(sv, i: int) -> float:
    if sv is None: return float("nan")
    total = 0.0; dim = len(sv.data); s = 0
    while s < dim:
        amp = sv.data[s]
        p = amp.real*amp.real + amp.imag*amp.imag
        total += p*(1.0 if _bit(s,i)==0 else -1.0)
        s += 1
    return total

def _particle_number_exp(sv, n_qubits: int) -> float:
    if sv is None: return float("nan")
    i = 0; acc = 0.0
    while i < n_qubits:
        acc += (1.0 - _z_exp_from_state(sv,i))*0.5
        i += 1
    return acc

def _random_params(n: int, seed: Optional[int]) -> List[float]:
    if seed is not None: random.seed(seed)
    out = []; i = 0
    while i < n:
        out.append(2.0*math.pi*random.random()); i += 1
    return out

def ansatz(params: List[float]):

    def G_gate(theta: float):
        qc2 = QuantumCircuit(2, name="G")
        qc2.cx(0,1); qc2.ry(theta*0.5, 0); qc2.cx(1,0); qc2.ry(-theta*0.5, 0); qc2.cx(1,0); qc2.cx(0,1)
        return qc2.to_gate()

    def cG_gate(theta: float):
        return G_gate(theta).control(1)

    if not isinstance(params, list): raise ValueError("params must be a list[float]")
    if len(params) < 6: raise ValueError("params must have length >= 6")

    try:
        qc = _build_hf_init(6, 3)  # |111000>
        qc.append(G_gate(params[0]), [2,3])
        qc.append(G_gate(params[1]), [3,4])
        qc.append(G_gate(params[2]), [4,5])
        qc.append(cG_gate(params[3]), [1,2,3])
        qc.append(cG_gate(params[4]), [1,3,4])
        qc.append(cG_gate(params[5]), [0,3,4])
        return qc
    except Exception as e:
        raise RuntimeError(f"Ansatz build failed: {e}")

# ---- Tests ----
def test_build(make_fn: Callable[[List[float]], "QuantumCircuit"]):
    try:
        params = _random_params(6, 0); qc = make_fn(params)
        from qiskit import QuantumCircuit as QC
        ok = isinstance(qc, QC) and qc.num_qubits == 6
        return ok, ("build: ok" if ok else "build: wrong type or qubit count")
    except Exception as e:
        return False, f"build: exception: {e}"

def test_zero_params_identity(make_fn):
    try:
        params = [0.0]*6
        hf = _build_hf_init(6, 3); qc = make_fn(params)
        sv_hf = _statevector_from_circuit(hf); sv_full = _statevector_from_circuit(qc)
        if sv_hf is None or sv_full is None: return False, "zero-params: simulation failed"
        d = _state_distance(sv_hf.data, sv_full.data)
        return (d < 1e-9), f"zero-params: distance={d:.3e}"
    except Exception as e:
        return False, f"zero-params: exception: {e}"

def test_param_influence(make_fn):
    try:
        base = [0.0]*6; base_qc = make_fn(base); sv0 = _statevector_from_circuit(base_qc)
        if sv0 is None: return False, "influence: base simulation failed"
        i = 0; ok = True; details = []
        while i < 6:
            p = [0.0]*6; p[i] = 1e-3
            sv = _statevector_from_circuit(make_fn(p))
            if sv is None: return False, "influence: simulation failed"
            d = _state_distance(sv0.data, sv.data)
            if d <= 1e-8: ok = False
            details.append(f"p{i}: Δ={d:.3e}"); i += 1
        return ok, "influence: " + ", ".join(details)
    except Exception as e:
        return False, f"influence: exception: {e}"

def test_periodicity(make_fn):
    try:
        base = _random_params(6, 123); sv_base = _statevector_from_circuit(make_fn(base))
        if sv_base is None: return False, "periodicity: base simulation failed"
        i = 0; ok = True; details = []
        while i < 6:
            shifted = base[:]; shifted[i] = shifted[i] + 2.0*math.pi
            sv_shift = _statevector_from_circuit(make_fn(shifted))
            if sv_shift is None: return False, "periodicity: simulation failed"
            d = _state_distance(sv_base.data, sv_shift.data)
            if d >= 1e-9: ok = False
            details.append(f"p{i}: dist={d:.1e}"); i += 1
        return ok, "periodicity: " + ", ".join(details)
    except Exception as e:
        return False, f"periodicity: exception: {e}"

def test_particle_number(make_fn):
    try:
        params = _random_params(6, 999); sv = _statevector_from_circuit(make_fn(params))
        if sv is None: return False, "particle-number: simulation failed"
        N = _particle_number_exp(sv, 6)
        return abs(N-3.0) < 1e-9, f"particle-number: <N>={N:.6f} (expect 3.000000 if conserved)"
    except Exception as e:
        return False, f"particle-number: exception: {e}"

def run_all_tests(make_fn):

    tests = [("build", test_build), ("zero_params_identity", test_zero_params_identity),
             ("param_influence", test_param_influence), ("periodicity", test_periodicity),
             ("particle_number", test_particle_number)]
    for name, fn in tests:
        ok, msg = fn(make_fn)
        print(f"[{'PASS' if ok else 'FAIL'}] {name}: {msg}")

# Run
run_all_tests(ansatz)


[PASS] build: build: ok
[PASS] zero_params_identity: zero-params: distance=0.000e+00
[FAIL] param_influence: influence: p0: Δ=5.000e-04, p1: Δ=1.633e-20, p2: Δ=1.633e-20, p3: Δ=5.000e-04, p4: Δ=1.633e-20, p5: Δ=1.633e-20
[FAIL] periodicity: periodicity: p0: dist=2.0e+00, p1: dist=3.3e-01, p2: dist=8.9e-02, p3: dist=2.0e+00, p4: dist=9.5e-01, p5: dist=9.5e-01
[PASS] particle_number: particle-number: <N>=3.000000 (expect 3.000000 if conserved)


## 2. Hamitonian
Take the given hamitonian and calculate its expectation value

## 3. Optimization
Optimize agianst $\theta$

## 4. Repeat
Repeat and output result